
# Hubble's law, Hubble constant, and the age of the Universe


Hubble's law is the observation that galaxies are moving away from
Earth at speeds proportional to their distances from Earth: 

$$v = H_0 \, D .$$

Here $H_0$ is the *Hubble constant*, $D$ is the distance to a
galaxy, and $v$ is the speed of separation.

Hubble constant is most frequently quoted in km/s/Mpc, thus giving
the speed in km/s of a galaxy 1 megaparsec away
(1Mpc = $3.09 \times 10^{19}$ km; the parsec, pc, is a unit of length 
that is used to measure the large distances to astronomical objects outside the Solar System).
However, the SI unit of $H_0$ is
simply $s^{-1}$. The reciprocal of $H_0$ is known as the
*Hubble time*. The Hubble time is the age the Universe would
have had if the expansion had been uniform in time; it is different from the
real age of the Universe because the real expansion is not
uniform. However, the Hubble time and the age of the Universe are
related by a dimensionless factor which depends on the mass-energy
content of the Universe; it assumed to be close to 0.96.


Here we determine the Hubble constant from
the experimental data on the *distance modulus* and *redshift* of supernovae.


Load the required packages:

In [ ]:

using CSV
using DataFrames
using PyPlot


The URL of the database of the measurements:

In [ ]:

url = "https://vizier.u-strasbg.fr/viz-bin/asu-txt?-source=J/ApJ/716/712/tableb2&-out=SN&-out=zCMB&-out=mu"


Download the data into a temporary file on the local machine:

In [ ]:

catalog = download(url);


Read the data from a disk into a DataFrame, skipping 38 lines of the header and giving the new names to the columns - `name`, `redshift`, and `modulus` for the name of the star, its red shift, and its distance modulus.

In [ ]:

df = CSV.read(catalog, DataFrame, skipto=38, delim=' ', ignorerepeated=true,
              types=[String, Float64, Float64], silencewarnings=true,
              header=["name", "redshift", "modulus"],)


Drop records with missing data:

In [ ]:

dropmissing!(df)


he *distance modulus* is a way of expressing distances that is often used in astronomy. 
It is a logarithmic measure of the distance to an astronomical object 
calculated from its apparent brightness and absolute brightness. The distance modulus, 
$\mu$, is related to the object's distance through the formula 

$$\mu = 5 \log_{10}(D) + 25,$$ 

where $D$ is the distance in Megaparsecs.


Let's define a helper function, `dist`, that given the modulus, calculates the distance in Mpc:

In [ ]:

dist(modulus) = 10.0 ^ (modulus / 5 - 5)


The main causes of electromagnetic redshift in astronomy and cosmology are the relative motions of radiation sources, 
which give rise to the *relativistic Doppler effect*, and the gravitational potentials, which gravitationally redshift escaping radiation. 

$$z \equiv \frac{\lambda_{\mathrm{ob}} - \lambda_{\mathrm{em}} }{\lambda_{\mathrm{em}}},$$

where $\lambda_{\mathrm{em}}$ the wavelength of emitted radiation and $\lambda_{\mathrm{ob}}$ is its observed wavelength.


The general formula for $z$ is rather complicated. However, for small redshifts, $z \ll 1$, it simplifies to

$$z \approx \frac{v}{c} ,$$

where $v$ is the speed of the star, and $c$ is the speed of light. 


$$z = \frac{v}{c} = \frac{H_0}{c}D .$$

The slope of the graph $z(D)$ gives the Hubble constant diveded by the speed of light.


Let's keep only the observational records where $z < 0.04$. 

In [ ]:

filter!(row -> row.redshift < 0.04, df);


Sort the observations by distance:

In [ ]:

sort!(df, "modulus");


Calculate distances in Mpc:

In [ ]:

distances = dist.(df.modulus);


Let's plot the **redshift vs distance** to the star:

In [ ]:

plot(distances, df.redshift, ".", label="measurements")
grid(true)
legend()
ylabel("Red Shift")
xlabel("Distance (Mpc)");


We observe a couple of outliers but the general dependence is consistent with the Hubble law, distance
is proportinal to redshift.


We use Least Squares Fit to find the parameters of the best linear regression.

In [ ]:

"""
   alpha, beta, sigma = linear_regression(x, y)

Least square linear regression fit y = alpha + beta * x
Sigma is standard deviation for beta
"""
function linear_regression(x, y)
    n = length(x)
    xbar = sum(x)/n
    ybar = sum(y)/n
    denom = sum((x .- xbar).^2)
    beta = sum((x .- xbar).*(y .- ybar))/denom
    alpha = ybar - beta*xbar
    sigma = sqrt(sum((y .- alpha .- beta*x).^2)/((n - 2)*denom))
    return alpha, beta, sigma
end

In [ ]:

alpha, beta, sigma = linear_regression(distances, df.redshift)


Plot of the result of the fit:

In [ ]:

plot(distances, df.redshift, ".", label="measurements")
plot(distances, alpha .+ beta*distances, label="LSq linear fit")
grid(true)
legend()
ylabel("Red Shift")
xlabel("Distance (Mpc)");


Hubble constant, in km/sec/Mpc:

In [ ]:

const c = 300000.0  # speed of light, km/sec
H0 = c * beta
round(H0, digits=3)


Standard deviation, in km/sec/Mpc:

In [ ]:

dH0 = c * sigma


Hubble constant in 1/sec:

In [ ]:

const mpc = 3.09e19     # 1 megaparsec in km
h0 = H0 / mpc
round(h0, sigdigits=3)


Hubble time in seconds:

In [ ]:

Th = 1 / h0
round(Th, sigdigits=3)


Hubble time in years:

In [ ]:

round(Th/(60*60*24*365), sigdigits=2)


Compare your values of the Hubble constant and the Hubble time with with the ones found in the literature: